In [1]:
import os
from importlib.resources import files
import time

import numpy as np
import torch
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

from learn_embedding.approximators import *
from learn_embedding.covariances import *
from learn_embedding.kernels import SquaredExp
from learn_embedding.embedding import Embedding
from learn_embedding.dynamics import FirstGeometry, SecondGeometry
from learn_embedding.utils import *
from learn_embedding.loss import *

In [2]:
dim = 2
num_traj = 3  # Number of trajectories
num_samples = 1000  # Desired number of samples per trajectory
first_order = False

train_x = np.empty((0, 2)) if first_order else np.empty((0, 4))
train_y = np.empty((0, 2))

for i in range(1, num_traj + 1):
    # Load trajectory data
    data = np.loadtxt("../learn_embedding/data/toycase/trajectory_" + str(i) + ".csv")
    
    # subsample
    idx = DataProcess.subsample(data[:,:2], num_samples)
    data[idx,:2] -= data[-1,:2]
    data[-1,2:4] = np.zeros(2)
    data[-1,-2:] = np.zeros(2)
    
    # add to datset
    train_x = np.append(train_x, data[idx,:2], axis=0)  if first_order else np.append(train_x, data[idx,:4], axis=0)
    train_y = np.append(train_y, data[idx,2:4], axis=0) if first_order else np.append(train_y, data[idx,-2:], axis=0)

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
train_x = torch.from_numpy(train_x).float().to(device).requires_grad_(True)
train_y = torch.from_numpy(train_y).float().to(device)

In [4]:
reps = 5
num_neurons = [8, 16, 32, 64, 128, 256]
num_layers = [1, 2, 3, 4, 5, 6]

counter = 1
loss_log = torch.zeros(len(num_layers), len(num_neurons), reps)

for k in range(reps):
    for i, l in enumerate(num_layers):
        for j, n in enumerate(num_neurons):
            if first_order:
                model = FirstGeometry(Embedding(FeedForward(dim, [n]*l, 1)), torch.tensor([0,0]).to(device), SPD(dim)).to(device)
            else:
                model = SecondGeometry(Embedding(FeedForward(dim, [n]*l, 1)), torch.tensor([0,0]).to(device), SPD(dim), SPD(dim)).to(device)
                
            trainer = Trainer(model, train_x, train_y)
        
            trainer.optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)
            # trainer.optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-2, weight_decay=1e-1)
            
            trainer.loss = torch.nn.MSELoss()
            # trainer.loss = torch.nn.SmoothL1Loss()
            
            trainer.options(normalize=False, shuffle=True, print_loss=False,epochs=2000)
            trainer.train()

            loss_log[i,j,k] = trainer.loss(model(train_x),train_y).item()
            print("Iter: ", counter, "Layers: ", l, "Neurons: ", n, "Loss: ", loss_log[i,j,k].item())
            counter += 1        

Iter:  1 Layers:  1 Neurons:  8 Loss:  0.0021159122698009014
Iter:  2 Layers:  1 Neurons:  16 Loss:  0.0005824918043799698
Iter:  3 Layers:  1 Neurons:  32 Loss:  0.0007172435289248824
Iter:  4 Layers:  1 Neurons:  64 Loss:  0.0009181457571685314
Iter:  5 Layers:  1 Neurons:  128 Loss:  0.0005858172080479562
Iter:  6 Layers:  1 Neurons:  256 Loss:  0.001829477259889245
Iter:  7 Layers:  2 Neurons:  8 Loss:  0.0012048723874613643
Iter:  8 Layers:  2 Neurons:  16 Loss:  0.0010371562093496323
Iter:  9 Layers:  2 Neurons:  32 Loss:  0.0008235651184804738
Iter:  10 Layers:  2 Neurons:  64 Loss:  0.001142196822911501
Iter:  11 Layers:  2 Neurons:  128 Loss:  0.0009353903005830944
Iter:  12 Layers:  2 Neurons:  256 Loss:  0.0016045888187363744
Iter:  13 Layers:  3 Neurons:  8 Loss:  0.00028444704366847873
Iter:  14 Layers:  3 Neurons:  16 Loss:  0.0005533546209335327
Iter:  15 Layers:  3 Neurons:  32 Loss:  0.0008019920205697417
Iter:  16 Layers:  3 Neurons:  64 Loss:  0.000653264403808862
It

Iter:  132 Layers:  4 Neurons:  256 Loss:  0.0006390867056325078
Iter:  133 Layers:  5 Neurons:  8 Loss:  0.0012595659354701638
Iter:  134 Layers:  5 Neurons:  16 Loss:  0.0010829002130776644
Iter:  135 Layers:  5 Neurons:  32 Loss:  0.0008401589002460241
Iter:  136 Layers:  5 Neurons:  64 Loss:  0.0011718636378645897
Iter:  137 Layers:  5 Neurons:  128 Loss:  0.004029595758765936
Iter:  138 Layers:  5 Neurons:  256 Loss:  0.0038688569329679012
Iter:  139 Layers:  6 Neurons:  8 Loss:  0.002128362189978361
Iter:  140 Layers:  6 Neurons:  16 Loss:  0.001752906246110797
Iter:  141 Layers:  6 Neurons:  32 Loss:  0.0014541366836056113
Iter:  142 Layers:  6 Neurons:  64 Loss:  0.0012721836101263762
Iter:  143 Layers:  6 Neurons:  128 Loss:  0.0011168880155310035
Iter:  144 Layers:  6 Neurons:  256 Loss:  0.0011297912569716573
Iter:  145 Layers:  1 Neurons:  8 Loss:  0.0005568850319832563
Iter:  146 Layers:  1 Neurons:  16 Loss:  0.0004182429111097008
Iter:  147 Layers:  1 Neurons:  32 Loss: 

In [5]:
loss_log.mean(dim=2)*1000

tensor([[1.7045, 1.1195, 1.0222, 0.7575, 0.8919, 1.2405],
        [0.9779, 1.1278, 0.8094, 0.9519, 1.1333, 2.6535],
        [0.9902, 0.6726, 0.9619, 0.6927, 1.3839, 3.7362],
        [1.1371, 1.1079, 1.4646, 1.4941, 3.5951, 4.0597],
        [1.1347, 2.2753, 1.3439, 1.2855, 1.7242, 4.7278],
        [1.5545, 1.7147, 1.8308, 1.2570, 2.4957, 5.7849]])

In [6]:
torch.argmin(loss_log.mean(dim=2))

tensor(13)

In [7]:
loss_log.std(dim=2)*1000

tensor([[1.1194, 0.9737, 0.6816, 0.1088, 0.4040, 0.4470],
        [0.3971, 0.4687, 0.3458, 0.3105, 0.9223, 1.9304],
        [0.5380, 0.2194, 0.3865, 0.3202, 0.3884, 2.4189],
        [0.2990, 0.4822, 0.4127, 1.3214, 2.7312, 2.4309],
        [0.3386, 1.7460, 0.5238, 0.3503, 1.3649, 2.3577],
        [0.6400, 0.2781, 1.5087, 0.2330, 2.0081, 2.7450]])

In [8]:
torch.argmin(loss_log.std(dim=2))

tensor(3)